In [2]:
# load libraries
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

import proteomics_downstream_analysis as pda
import os

from functools import reduce

In [2]:
# pd.set_option('display.max_rows', 10)


In [11]:
data = pd.DataFrame({
            'Protein.Ids': ['P123', 'P234', 'P345', 'P456', 'P567',
                            'P232', 'P124', 'P214', 'P352', 'P109'],
            'WT1': [1.0, 1.0, 1, 1.0, 1.0,
                    1.0, 1.0, 1, 1.0, 1.0],
            'WT2': [1.0, 1.0, 1.0, 1.0, 1.0,
                    1.0, 1.0, 1, 1.0, 1.0],
            'WT3': [1.0, 1.0, 1.0, 1.0, 1.0,
                    1.0, 1.0, 1, 1.0, 1.0],
            'KO1': [1.0, 1.0, 1.0, 1.0, 1.0,
                    1.0, 1.0, 1, 1.0, 1.0],
            'KO2': [1.0, 1.0, 1.0, 1.0, 1.0,
                    1.0, 1.0, 1, 1.0, 1.0],
            'KO3': [1.0, 1.0, 1.0, 1.0, 1.0,
                    1.0, 1.0, 1, 1.0, 1.0],   
        })
data = obj.paralell_processing(data, dummy_function)


In [4]:
# read data
filepath = 'bl_data.csv'
data = pd.read_csv(filepath)
data['Genes'] = data['Genes'].astype('string')
data['Protein.Ids'] = data['Protein.Ids'].astype('string')

# read the contamination panel
filepath = '/Users/tungvuduc/Desktop/PhD/projects/PPMI/urine_analysis/contam_panel.xlsx'
panel = pd.read_excel(filepath)


In [5]:
# read meta data
# # Path
folder = "/Users/tungvuduc/Desktop/PhD/projects/PPMI/meta_data/meta_data_for_analysis/feature_engineering_meta_data"

files = {os.path.splitext(file)[0]: pd.read_csv(os.path.join(folder, file)) 
              for file in os.listdir(folder) if file.endswith('.csv') 
              and not file.startswith('code_decode')}

In [6]:
# change patno to string
for key in files.keys():
    files[key]['PATNO'] = files[key]['PATNO'].astype('string')
    
list(files.keys())

['educ_data',
 'schwab_england_data',
 'updrs_data',
 'age_data_long',
 'demo_data',
 'disease_state',
 'participant_info',
 'gen_data',
 'updrs_data_long',
 'fam_hist_data',
 'age_data',
 'schwab_england_data_long']

In [7]:
# create tables with different meta data
# merge cohort and mutation columns
cohort_mut_meta = files['disease_state'].merge(files['participant_info'], on='PATNO')
cohort_mut_meta['cohort_lrrk2_mutation'] = cohort_mut_meta['disease_state'] + ' & ' + cohort_mut_meta['LRRK2 mutation']
cohort_mut_meta['cohort_gba_mutation'] = cohort_mut_meta['disease_state'] + ' & ' + cohort_mut_meta['GBA mutation']

# create pda object and add data
obj = pda.DiannData()

for i in cohort_mut_meta.columns[1:]:
    obj.data = data.copy()
    obj.update_col_names(cohort_mut_meta, 'PATNO', i)
    obj.datasets.append(obj.data)

In [21]:
# merge meta data
# impute missing data in age meta data
patno_age_data = files['age_data']['PATNO'].tolist()
patno = data.columns[2:].tolist()

missing_patno = [i for i in patno if i not in patno_age_data]
print(f'These patno are missing: {missing_patno}')

files['age_data'].loc[978] = ['3069', 'BL', np.mean(files['age_data']['AGE_AT_VISIT'])]
files['age_data'].loc[979] = ['56886', 'BL', np.mean(files['age_data']['AGE_AT_VISIT'])]

# merge meta data
datasets = [files['age_data'][['PATNO', 'AGE_AT_VISIT']],
            files['demo_data'][['PATNO','sex']], 
            cohort_mut_meta,
            files['updrs_data'][files['updrs_data']['Visit ID'] == 'BL'].drop_duplicates(subset = 'PATNO')]

meta_data = reduce(lambda x, y: pd.merge(x, y, on='PATNO', how='outer'), datasets)

meta_data = meta_data.rename(columns={'AGE_AT_VISIT': 'age'})

meta_data = meta_data.set_index('PATNO').reindex(data.columns[2:])
obj.meta_data = meta_data.copy()

These patno are missing: ['3069', '56886']


In [22]:
# print unique columns in each dataset
for idx, df in enumerate(obj.datasets):
    print(f'Dataset {idx + 1}: {df.columns.unique().tolist()[2:]}')

Dataset 1: ['PD', 'Prodromal', 'Control']
Dataset 2: ['LRRK2+', 'LRRK2-']
Dataset 3: ['GBA-', 'GBA+']
Dataset 4: ['LRRK2+&GBA-', 'LRRK2+&GBA+', 'LRRK2-&GBA-', 'LRRK2-&GBA+']
Dataset 5: ['PD & LRRK2+', 'Prodromal & LRRK2+', 'Control & LRRK2-', 'PD & LRRK2-', 'Prodromal & LRRK2-']
Dataset 6: ['PD & GBA-', 'Prodromal & GBA-', 'Prodromal & GBA+', 'PD & GBA+', 'Control & GBA-', 'Control & GBA+']


In [9]:
# preprocessing
datasets = obj.datasets.copy()
obj.datasets = []
names = ['cohort', 'lrrk2 mutation', 'gba mutation', 'lrrk2 and gba mutation', 'cohort and lrrk2 mutation', 'cohort and gba mutation']

for name, dataset in zip(names[0:2], datasets[:2]):
    obj.data = dataset.copy()
    obj.preprocessing(method='hybrid',
                     completeness=0.5,
                     percentage=0.8,
                     strategy='mean',
                     kind='knn'
                     )
    obj.add_data(obj.data, name)

Data with the title "cohort" is added
Total number of datasets: "1"
Data with the title "lrrk2 mutation" is added
Total number of datasets: "2"


In [10]:
# remove rbc and zscore outliers
zscore_outliers = [obj.outlier(dataset,'zscore', False)[1] for dataset in obj.datasets] 
rbc_outliers = [obj.outlier(dataset,'contamination', False, panel)[1] for dataset in obj.datasets]

# remove outliers
outliers = [(zscore_outliers[i] + rbc_outliers[i]) for i in range(len(obj.datasets))]

# get inliers
inliers = [~(outlier) for outlier in outliers]

# remove outliers from dataset
obj.datasets = [dataset.set_index(['Protein.Ids', 'Genes']).iloc[:, inlier] for dataset, inlier in zip(obj.datasets, inliers)]

In [11]:
# sort meta data
meta_data_cohort = obj.meta_data.sort_values('disease_state')
meta_data_lrrk2_mut = obj.meta_data.sort_values('LRRK2 mutation')
meta_data_gba_mut = obj.meta_data.sort_values('GBA mutation')
meta_data_lrrk2_gba_mut = obj.meta_data.sort_values('mutation')
meta_data_cohort_lrrk2_mut = obj.meta_data.sort_values('cohort_lrrk2_mutation')
meta_data_cohort_gba_mut = obj.meta_data.sort_values('cohort_gba_mutation')

#collect meta data
meta_datasets = [meta_data_cohort, 
                 meta_data_lrrk2_mut, 
                 meta_data_gba_mut,
                 meta_data_lrrk2_gba_mut,
                 meta_data_cohort_lrrk2_mut,
                 meta_data_cohort_gba_mut]

# remove outliers in meta data
obj.meta_datasets = [dataset.iloc[inlier] for dataset, inlier in zip(meta_datasets, inliers)]

# one hot encoding of meta data
renamer = {'GBA+':1, 'GBA-':0,
           'LRRK2+':1, 'LRRK2-':0}
obj.meta_datasets = [dataset.replace(renamer) for dataset in obj.meta_datasets]
obj.meta_datasets = [i.reset_index(drop=True) for i in obj.meta_datasets]

### Ancova for Control vs Prodromal | Control vs PD

In [12]:
dataset = obj.datasets[0].reset_index()
cov_data = obj.meta_datasets[0]
groups = [['Control', 'Prodromal'],
          ['Control', 'PD']]

sample_col = 'disease_state'
cov = ['sex', 'age']
a, b, results = obj.two_tailed_ancova(dataset, cov_data, groups, sample_col, cov)

100%|██████████| 377/377 [00:04<00:00, 92.06it/s] 


In [174]:
import pingouin as pg
from statsmodels.stats.multitest import fdrcorrection

# create a test_dataset with 4 columns, with the name ['age' 'sex', 'sample', 'ProteinId_1', 'ProteinId_2']
data = pd.DataFrame({
    'age': [10, 12, 13, 16, 12, 18, 12, 13, 16],
    'sex': [0, 1, 1, 1, 0, 1, 0, 1, 1],
    'sample': ['Control', 'Control', 'Control',
               'Disease1', 'Disease1', 'Disease1',
               'Disease2', 'Disease2', 'Disease2'],
    'ProteinId_1': [123., 124., 128., 129., 194., 283., 290., 290., 290.,],
    'ProteinId_2': [123., 134., 188., 199., 154., 273., 210., 210., 210.,],
    'ProteinId_3': [173., 134., 148., 179., 124., 283., 220., 260., 220.,],
    'ProteinId_4': [163., 144., 128., 119., 124., 283., 210., 200., 260.,],
    'ProteinId_5': [183., 154., 178., 129., 154., 283., 200., 220., 270.,],
    'ProteinId_6': [123., 124., 128., 129., 194., 283., 290., 290., 290.,],
    'ProteinId_7': [123., 134., 188., 199., 154., 273., 210., 210., 210.,],
    'ProteinId_8': [173., 134., 148., 179., 124., 283., 220., 260., 220.,],
    'ProteinId_9': [163., 144., 128., 119., 124., 283., 210., 200., 260.,],
    'ProteinId_10': [183., 154., 178., 129., 154., 283., 200., 220., 270.,],
})

results = [pg.ancova(data=data, dv=i, covar=['age', 'sex'], between='sample').iloc[0] for i in data.filter(regex='ProteinId_').columns.tolist()]

result = pd.DataFrame(results)
pvalues = -np.log10(result['p-unc'])
pvalues.tolist()

groups = [['Control', 'Disease1'],
          ['Control', 'Disease2']]

pvalues = []
qvalues = []
for group in groups:
    results = [pg.ancova(data=data[data['sample'].isin(group)], dv=i, covar=['age', 'sex'], between='sample').iloc[0] for i in data.filter(regex='ProteinId_').columns.tolist()]

    result = pd.DataFrame(results)
    pvalue = -np.log10(result['p-unc'])
    pvalues.append(np.array(pvalue))
    qvalues.append(fdrcorrection(result['p-unc'])[1])


In [175]:
pvalues = np.concatenate(pvalues, axis=0)
qvalues = np.concatenate(qvalues, axis=0)
qvalues

array([0.65776381, 0.371654  , 0.30992508, 0.34133167, 0.34133167,
       0.65776381, 0.371654  , 0.30992508, 0.34133167, 0.34133167,
       0.00138658, 0.36977433, 0.19025848, 0.36977433, 0.52634941,
       0.00138658, 0.36977433, 0.19025848, 0.36977433, 0.52634941])

In [134]:
dataset = data.iloc[:, 2:].set_index('sample').T.reset_index(names='Protein.Ids')
dataset['Genes'] = dataset['Protein.Ids']
cov_data = data.iloc[:, :2]

cov = ['age', 'sex']

pvals, qvals, results = obj.two_tailed_ancova(dataset, cov_data, groups, 'sample', cov)

100%|██████████| 1/1 [00:00<00:00, 122.82it/s]


In [4]:
import pandas as pd
import proteomics_downstream_analysis as pda

data1 = pd.DataFrame({
            'Protein.Ids': ['P123', 'P234', 'P345', 'P456', 'P567',
                            'P232', 'P124', 'P214', 'P352', 'P109'],
            'WT1': [1.0, 1.0, 1, 1.0, 1.0,
                    1.0, 1.0, 1, 1.0, 1.0],
            'WT2': [1.0, 1.0, 1.0, 1.0, 1.0,
                    1.0, 1.0, 1, 1.0, 1.0],
            'WT3': [1.0, 1.0, 1.0, 1.0, 1.0,
                    1.0, 1.0, 1, 1.0, 1.0],
            'KO1': [1.0, 1.0, 1.0, 1.0, 1.0,
                    1.0, 1.0, 1, 1.0, 1.0],
            'KO2': [1.0, 1.0, 1.0, 1.0, 1.0,
                    1.0, 1.0, 1, 1.0, 1.0],
            'KO3': [1.0, 1.0, 1.0, 1.0, 1.0,
                    1.0, 1.0, 1, 1.0, 1.0],   
        })

def dummy_function(data):
    return data.select_dtypes(float)
obj = pda.DiannData()

pd.concat(obj.paralell_processing(data1, dummy_function), axis=0).shape

(10, 6)

In [1]:
from test_parallelprocessing import TestParallelProcessing as ts

obj = ts()
obj.setUp()

obj.test_split_data_for_parallel_processing()